In [1]:
import csv

In [2]:
def create_typo_per_ego():
    main_dir = 'facebook_k5'
    typo_per_ego_per_k = {}
    for classe in range(2,16):
        for temp_classe in range(classe):
            with open('../Results/%s/Typo_%s/egos_classe_%s.csv' % (main_dir, classe, temp_classe), 'r') as to_read:
                csv_r = csv.reader(to_read, delimiter = ';')
                en_tete = csv_r.next()
                for line in csv_r:
                    ego = line[0]
                    if not ego in typo_per_ego_per_k:
                        typo_per_ego_per_k[ego] = {}
                    typo_per_ego_per_k[ego][int(classe)] = int(temp_classe)
    
    with open('../Results/%s/typo_par_ego.csv' % main_dir, 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ';')
        csv_w.writerow(['ego'] + ['k = %s' % temp_classe for temp_classe in range(2,16)])
        for ego in typo_per_ego_per_k:
            csv_w.writerow([ego] + [typo_per_ego_per_k[ego][classe] for classe in range(2,16)])

def create_all_rel_sankey():
    
    suivants_par_cluster = {}
    size_par_cluster = {}
    
    with open(MAIN_DIR+'/typo_par_ego.csv', 'r') as to_read:
        csv_r = csv.reader(to_read, delimiter = ';')
        
        first_line = csv_r.next()
        nb_col = len(first_line) - 1
        
        for line in csv_r:
            id_premier_groupe = 2
            for elem in line[id_premier_groupe-1:]:
                if not '%s_%s' % (id_premier_groupe, elem) in suivants_par_cluster:
                    size_par_cluster['%s_%s' % (id_premier_groupe, elem)] = 1
                    suivants_par_cluster['%s_%s' % (id_premier_groupe, elem)] = {}
                else:
                    size_par_cluster['%s_%s' % (id_premier_groupe, elem)] += 1
                id_second_groupe = id_premier_groupe + 1
                suivants_groupe_1 = suivants_par_cluster['%s_%s' % (id_premier_groupe, elem)]
                for elem_2 in line[id_second_groupe-1:]:
                    if not '%s_%s' % (id_second_groupe, elem_2) in suivants_groupe_1:
                        suivants_groupe_1['%s_%s' % (id_second_groupe, elem_2)] = 1
                    else:
                        suivants_groupe_1['%s_%s' % (id_second_groupe, elem_2)] += 1
                    id_second_groupe += 1
                id_premier_groupe += 1
                
    liste_clusters_target = set()
    
    for cluster_origine in suivants_par_cluster:
        keys = suivants_par_cluster[cluster_origine]
        for key in keys:
            liste_clusters_target.add(key)
            
    liste_clusters_target = list(liste_clusters_target)
    liste_clusters_target.sort(key = lambda x : (int(x.split('_')[0]), int(x.split('_')[1])))
        
    liste_clusters_source = [x for x in suivants_par_cluster.keys() if not suivants_par_cluster[x] == {}]
    liste_clusters_source.sort()
    
            
    with open(MAIN_DIR+'/all_rel_sankey.csv', 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ';')
        first_line = []
        k_former_target = 2
        for cluster_target in liste_clusters_target:
            k_this_target = int(cluster_target.split('_')[0])
            if k_this_target > k_former_target:
                first_line.append('')
            first_line.append(cluster_target.split('_')[0]+'_'+str(int(cluster_target.split('_')[1])+1))
            k_former_target = k_this_target
        
        csv_w.writerow(first_line)
        
        k_former_source = 2
        
        for source in liste_clusters_source:            
            line_to_write = [source.split('_')[0]+'_'+str(int(source.split('_')[1])+1)]
            k_this_source = int(source.split('_')[0])
            if k_this_source > k_former_source:
                csv_w.writerow([])
            k_former_target = 3
            
            for target in liste_clusters_target:
                k_this_target = int(target.split('_')[0])
                if k_this_target > k_former_target:
                    line_to_write.append('')           
                if k_this_target <= k_this_source:
                    line_to_write.append('')
                    k_former_target = k_this_target
                    continue
                k_former_target = k_this_target
                percent = round(100*suivants_par_cluster[source].get(target, 0) / float(size_par_cluster[source]), 1)
                line_to_write.append(percent)
            
            csv_w.writerow(line_to_write)
            k_former_source = k_this_source
            
    with open('sankey.html', 'r') as to_read:
        with open(MAIN_DIR+'/sankey.html', 'w') as to_write:
            for line in to_read:
                to_write.write(line)

In [3]:
def create_all_sankey():
    sources_per_target, targets_per_source, size_per_target, size_per_source = read_typos()
    list_sources = targets_per_source.keys()
    list_sources.sort(key = lambda x : (int(x.split('_')[0]), int(x.split('_')[1])))
    with open(MAIN_DIR+'/all_rel_sankey.csv', 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ',')
        csv_w.writerow(['source','target','value'])

        for source in list_sources:            
            for target in targets_per_source[source]:       
                csv_w.writerow([source.split('_')[0]+'_'+str(int(source.split('_')[1])),
                                target.split('_')[0]+'_'+str(int(target.split('_')[1])),
                                sources_per_target[target][source]])

    with open('sankey.html', 'r') as to_read:
        with open(MAIN_DIR+'/sankey.html', 'w') as to_write:
            for line in to_read:
                to_write.write(line) 


In [30]:
def read_typos():
    sources_per_target = {}
    size_per_target = {}
    targets_per_source = {}
    size_per_source = {}
    
    with open(MAIN_DIR+'/typo_par_ego.csv', 'r') as to_read:
        csv_r = csv.reader(to_read, delimiter = ';')
        csv_r.next()
        
        for line in csv_r:
            for k_source in range(1,len(line)-1):                
                source = '%s_%s' % (k_source+1, line[k_source])
                target = '%s_%s' % (k_source+2, line[k_source+1])
                
                sources_per_target[target] = sources_per_target.get(target, {})
                targets_per_source[source] = targets_per_source.get(source, {})
                size_per_target[target] = size_per_target.get(target, 0) + 1
                size_per_source[source] = size_per_source.get(source, 0) + 1
                
                sources_per_target[target][source] = sources_per_target[target].get(source, 0) + 1
                targets_per_source[source][target] = targets_per_source[source].get(target, 0) + 1
    
    return sources_per_target, targets_per_source, size_per_target, size_per_source
 
    
def create_sankey():

    sources_per_target, targets_per_source, size_per_target, size_per_source = read_typos()
    list_sources = targets_per_source.keys()
    list_sources.sort(key = lambda x : (int(x.split('_')[0]), int(x.split('_')[1])))
    with open(MAIN_DIR+'/sankey.csv', 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ',')
        csv_w.writerow(['source','target','value'])
        
        for source in list_sources:            
            for target in targets_per_source[source]:       
                csv_w.writerow([source.split('_')[0]+'_'+str(int(source.split('_')[1])),
                                target.split('_')[0]+'_'+str(int(target.split('_')[1])),
                                sources_per_target[target][source]])
                
    with open('sankey.html', 'r') as to_read:
        with open(MAIN_DIR+'/sankey.html', 'w') as to_write:
            for line in to_read:
                to_write.write(line)
                

def percent_sources_targets(sources_per_target, targets_per_source, size_per_target, size_per_source):
    list_targets = sources_per_target.keys()
    list_targets.sort(key = lambda x : (int(x.split('_')[0]), int(x.split('_')[1])), reverse = True)
    
    percent_sources_per_target = {}
    for target in list_targets:
        percent_sources_per_target[target] = {}
        for source in sources_per_target[target]:
            percent_sources_per_target[target][source] = 100*sources_per_target[target][source] / float(size_per_target[target])
    
    list_sources = targets_per_source.keys()
    list_sources.sort(key = lambda x : (int(x.split('_')[0]), int(x.split('_')[1])))
    percent_targets_per_source = {}
    
    for source in list_sources:
        percent_targets_per_source[source] = {}
        for target in targets_per_source[source]:
            percent_targets_per_source[source][target] = 100*targets_per_source[source][target] / float(size_per_source[source])

    return list_sources, targets_per_source, percent_sources_per_target, percent_targets_per_source
            
def create_simple_sankey():
    sources_per_target, targets_per_source, size_per_target, size_per_source = read_typos()
    
    list_sources, targets_per_source, \
    percent_sources_per_target, percent_targets_per_source = \
        percent_sources_targets(sources_per_target, targets_per_source, size_per_target, size_per_source)
    sources_ok = ['2_0', '2_1']
    was_already_coherent = []
    coherence_before_coherence = {}
    
    with open(MAIN_DIR+'/simple_sankey.csv', 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ',')
        csv_w.writerow(['source','target','value'])
        
        for source in list_sources:
            #Si la source n'est pas dispo
            #on ne la calcule pas
            if not source in sources_ok: 
                continue
                
            link_exists, link_is_all = False, False    
            for target in targets_per_source[source]:
                lvl = int(target.split('_')[0])-2
                
                v_same = max(65, VALUE_SAME * (1 - lvl/10.))
                v_min = VALUE_MIN_EXISTS
                
                #Si il y a moins de x% de passage de l'un à l'autre, on la trouvera autrement 
                #ou pas
                if percent_targets_per_source[source][target] < VALUE_MIN_EXISTS:
                    continue
                    
                #Si ça ne suffit pas pour s'arrêter on le note dans les sources potentielles
                if (percent_sources_per_target[target][source] > v_same \
                    and percent_targets_per_source[source][target] > v_same):
                        gchild_ok = False
                        if not target in percent_targets_per_source:
                            gchild_ok = True
                            continue
                        for gtarget in percent_targets_per_source[target]:
                            if (percent_targets_per_source[target][gtarget] > v_same \
                                and percent_targets_per_source[source][target] > v_same):
                                gchild_ok = True
                        if gchild_ok:
                            continue
                
                sources_ok.append(target)   
                csv_w.writerow([source.split('_')[0]+'_'+str(int(source.split('_')[1])),
                            target.split('_')[0]+'_'+str(int(target.split('_')[1])),
                            sources_per_target[target][source]])

    with open('simple_sankey.html', 'r') as to_read:
        with open(MAIN_DIR+'/simple_sankey.html', 'w') as to_write:
            for line in to_read:
                to_write.write(line)

In [32]:
VALUE_SAME = 100.0
VALUE_MIN_EXISTS = 30.0

print [max(60, VALUE_SAME * (1 - lvl/10.)) for lvl in range (10)]

create_simple_sankey()

[100.0, 90.0, 80.0, 70.0, 60, 60, 60, 60, 60, 60]


In [6]:
corpus = 'facebook'
k = 5

FILE_NAME = '%s_k%s' % (corpus, k)
MAIN_DIR = '../Results/%s' % FILE_NAME

In [ ]:
create_typo_per_ego()
create_all_rel_sankey()
create_sankey()

In [24]:
def create_dir(k_value):
    directory = '%s/Typo_%s' % (MAIN_DIR, k_value)
    if not 'Typo_%s' % k_value in os.listdir(MAIN_DIR):
        os.mkdir(directory)
    return directory

def get_data(file_name = None, representativity = True):
    if not file_name:
        file_in = '../Data/%s.csv' % FILE_NAME
    else:
        file_in = file_name
    egos, typo_vector = [], {}
    nb_par_ego = {}
    with open(file_in, 'r') as to_read:
        csv_r = csv.reader(to_read, delimiter = ';')
        
        first_line = csv_r.next()
        keys_typo = [int(x.split('_')[-1]) for x in first_line[1:len(first_line)]]
        
        for line in csv_r:
            ego = line[0]
            if ego in egos:
                ego = ego + '_%s' % (nb_par_ego.get(ego, 0) + 1)
                nb_par_ego[ego] = 1 + nb_par_ego.get(ego, 0)
            egos.append(ego)
            if not representativity:
                typo_vector[ego] = {int(keys_typo[i]) : float(line[i+1]) for i in range(len(keys_typo))}
            else:
                typo_vector[ego] = \
                {
                    int(keys_typo[i]) : float(line[i+1]) if float(line[i+1]) <= 1 \
                                        else 2-1/float(line[i+1])

                    for i in range(len(keys_typo))
                }
                    
                
    return egos, typo_vector, keys_typo

def get_overall_representativity():
    s = {x : 0 for x in DATA_MOTIFS[DATA_MOTIFS.keys()[0]]}     
       
    for ego in DATA_MOTIFS:    
        for graphlet in DATA_MOTIFS[ego]:
            s[graphlet] += int(DATA_MOTIFS[ego][graphlet])
    
    all_graphlets = sum(s.values())
    return {x : s[x]/float(all_graphlets) for x in s}


def get_classe_representativity(list_of_egos):
    s = {x : 0 for x in DATA_MOTIFS[DATA_MOTIFS.keys()[0]]}
    for ego in list_of_egos:
        for graphlet in DATA_MOTIFS[ego]:
            s[graphlet] += int(DATA_MOTIFS[ego][graphlet])
    
    all_graphlets = float(sum(s.values()))
    local_representativities = {x : s[x]/all_graphlets for x in s}
    
    return {x : local_representativities[x] / float(OVERALL_REPR[x]) \
            if local_representativities[x] / float(OVERALL_REPR[x]) < 1
            else
                2 - 1 / (local_representativities[x] / float(OVERALL_REPR[x]))
            for x in local_representativities}

In [33]:
import os


corpus = 'facebook'
k = 5

FILE_NAME = '%s_k%s' % (corpus, k)
MAIN_DIR = '../Results/%s' % FILE_NAME

create_sankey()
create_simple_sankey()

clusters = []

#On garde les derniers clusters de chaque chaine
with open('%s/simple_sankey.csv' % MAIN_DIR, 'r') as to_read:
    csv_r = csv.reader(to_read, delimiter = ',')
    csv_r.next()
    for line in csv_r:
        if not line[1] in clusters:
            clusters.append(line[1])
        if line[0] in clusters:
            clusters.remove(line[0])
            
clusters.sort(reverse = True, key = lambda x : x.split('_')[0])
egos, DATA_MOTIFS, keys = get_data('../Data/motifs_%s.csv' % FILE_NAME)
DATA_MOTIFS = get_data('../Data/motifs_%s.csv' % FILE_NAME, representativity = False)[1]
OVERALL_REPR = get_overall_representativity()

egos_ok = []
clusters_values, clusters_size = {}, {}

for cluster in clusters:
    size, classe = [int(x) for x in cluster.split('_')]
    
    with open('%s/Typo_%s/egos_classe_%s.csv' % (MAIN_DIR, size, classe)) as to_read:
        csv_r = csv.reader(to_read, delimiter = ';')
        csv_r.next()
        list_egos = []
        for line in csv_r:
            ego = line[0]
            if not ego in egos_ok:
                list_egos.append(ego)
                egos_ok.append(ego)
        print cluster
        clusters_values[cluster] = get_classe_representativity(list_egos)
        clusters_size[cluster] = len(list_egos)
        
with open('%s/Typo_%s/kmeans_stats.csv' % (MAIN_DIR, 'sankey'), 'w') as to_write:
    csv_w = csv.writer(to_write, delimiter = ';')
    csv_w.writerow(['classe', 'nb',''] + keys)
    for cluster in clusters_values:
        csv_w.writerow([cluster, clusters_size[cluster],''] + [clusters_values[cluster][k] for k in keys])

6_4
6_2
5_1
5_2
4_3
4_1
